# ABS Retail Trade 8501

Note: the ABS usually releases the Retail Trade data in tranches, with the second tranch delivered about a week after the first.

## Python set-up

In [1]:
# system imports
from typing import cast
import textwrap

# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_abs_data
from abs_plotting import fix_abs_title
from mgplot import (
    line_plot_finalise,
    postcovid_plot_finalise,
    series_growth_plot_finalise,
    finalise_plot,
    multi_start,
)
from decompose import decompose

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# constants
LONG_LABEL = 45

# display charts in the notebook ...
SHOW = False

Could not import auto_arima from pmdarima


## Get data from the ABS

### Retail trade data

In [2]:
abs_dict, meta, source, _ = get_abs_data("8501.0")

In [3]:
# Tables 1 to 4 are monthly tables; Table 5 and following are quarterly
print(textwrap.fill(", ".join(list(abs_dict.keys())), 110))

850101, 850102, 850103, 850104, 850105, 850106, 850107, 850108, 850109, 8501010, 8501011, 8501012, 8501013,
8501014, 8501015, 8501016, 8501019, 8501020, Table_21_Online_Retail_Turnover_Australia_by_type_of_Activity,
Table_22_Online_Retail_Turnover_Australia_by_type_of_Activity_Percentage_Change,
Table_23_Online_Retail_Turnover_Australia_by_type_of_Activity_Percentage_of_Total_Australian_Retail_turnover


In [4]:
# Check latest monthly date
print(abs_dict["850101"].index[-1])

2025-04


In [5]:
# check latest quarterly date
if "850105" in abs_dict:
    print(abs_dict["850107"].index[-1])

2025Q1


### CPI deflator

In [6]:
def get_cpi_deflator() -> tuple[pd.Series, str, str]:
    """Get CPI deflator, including a forward projection."""

    cat, table = "6401.0", "640106"
    cpi, _mcpi = ra.read_abs_series(
        cat=cat, series_id="A2325846C", single_excel_only=table
    )

    # rebase quarterly CPI index to one
    cpi_q = cpi[cpi.columns[0]] / cpi[cpi.columns[0]].iloc[-1]

    # calculate recent monthly CPI growth
    months_per_quarter = 3
    quarters = 2
    period = months_per_quarter * quarters  # in months
    monthly_growth = (cpi_q.iloc[-1] / cpi_q.iloc[-1 - quarters]) ** (1 / period)

    # create a monthly CPI index ...
    cpi_m = ra.qtly_to_monthly(cpi_q)

    # extrapolate monthly index forward ...
    additional_months = 4
    for _ in range(additional_months):
        cpi_m[cpi_m.index[-1] + 1] = cpi_m.iloc[-1] * monthly_growth

    print()
    print(f"Monthly growth: {monthly_growth}")
    print(f"Projected CPI: {cpi_m.tail(12)}")
    return cpi_m, cat, table

In [7]:
CPI_DEFLATOR, CPI_CAT_ID, CPI_TABLE = get_cpi_deflator()


Monthly growth: 1.0019079620461868
Projected CPI: Series ID
2024-08    0.987918
2024-09    0.988628
2024-10    0.989339
2024-11    0.990050
2024-12    0.990760
2025-01    0.993840
2025-02    0.996920
2025-03    1.000000
2025-04    1.001908
2025-05    1.003820
2025-06    1.005735
2025-07    1.007654
Freq: M, Name: A2325846C, dtype: float64


## Plot - retail turnover by industry

### Main plotting loop

In [12]:
def produce_charts() -> None:
    """Produce charts."""

    series_type = "Seasonally Adjusted"
    rows = meta[
        meta[mc.did].str.contains("Turnover", regex=False)
        & meta[mc.unit].str.contains("$", regex=False)
        & (meta[mc.stype] == series_type)
    ]

    for _row_index, row in rows.iterrows():
        # get the data for wacg plotable item
        series_id, units, table = row[mc.id], row[mc.unit], row[mc.table]
        data = abs_dict[table]
        series, units = ra.recalibrate(data[series_id], units)
        series_freq = cast(pd.PeriodIndex, series.index).freqstr
        check = pd.Period("2017-01-01", freq=series_freq)
        check_series = series[series.index > check]
        if len(check_series) != len(check_series.dropna()):
            print(f"Odd or missing data: {row[mc.did]}")
            continue

        # plot the data
        headline = "!"
        pre_tag = headline if table in ["850101", "850107"] else ""
        series.name = "Series"
        monthly = "Monthly"
        frequency = {"Q": "Quarterly", "M": monthly}[series_freq[0]]
        lfooter = f"Australia. {series_type.capitalize()}. "
        current_prices = "Current Prices. "
        lfooter += "Current Prices. " if frequency == monthly else ""
        title, lfooter = fix_abs_title(row[mc.did], lfooter)
        separator = "\n" if len(title) > LONG_LABEL else " "
        title = (
            f"{frequency} Retail Trade:{separator}"
            # Move "Turnover" to the y-axis label.
            f"{title.replace('Turnover', '').strip()}"
        )

        # Only do charts for the headline items
        if pre_tag != headline:
           continue

        # --- line plot of series
        line_plot_finalise(
            series,
            title=title,
            ylabel=f"Turnover {units}",
            pre_tag=pre_tag,
            rfooter=f"{source} {table}",
            lfooter=lfooter,
            annotate=True,
            show=SHOW,
        )

        # --- recent growth
        series_growth_plot_finalise(
            series,
            title=f"Growth in {title.title()}",
            plot_from=-19,
            pre_tag=pre_tag,
            rfooter=f"{source}",
            lfooter=lfooter,
            show=pre_tag == headline and SHOW,
        )

        # --- CPI adjusted mpnthly series ---
        if frequency == monthly:
            cpi_adj_series = (series / CPI_DEFLATOR).dropna()
            cpi_adj_series.name = "series"
            postcovid_plot_finalise(
                cpi_adj_series,
                title=f"{title} (CPI Adjusted)",
                ylabel=f"Turnover {units} (CPI Adj)",
                pre_tag=pre_tag,
                tag="cpi-adjusted",
                rfooter=f"{source} {CPI_CAT_ID}",
                lfooter=lfooter.replace(current_prices, "CPI adjusted prices. "),
                lheader="CPI Adjusted using All Groups CPI",
                show=pre_tag == headline and SHOW,
            )

            series_growth_plot_finalise(
                cpi_adj_series,
                title=f"Growth in {title.title()} (CPI Adjusted)",
                plot_from=-19,
                pre_tag=pre_tag,
                rfooter=f"{source}",
                lfooter=lfooter,
                show=pre_tag == headline and SHOW,
            )

        # --- Quarterly growth ---
        if frequency != monthly:
            multi_start(
                series,
                function=series_growth_plot_finalise,
                starts=[0,],
                title=f"Growth in {title.title()}",
                pre_tag=headline,
                rfooter=f"{source}",
                lfooter=lfooter,
                show=SHOW,
            )


produce_charts()

Odd or missing data: Turnover ;  Queensland ;  Liquor retailing ;
Odd or missing data: Turnover ;  Queensland ;  Other specialised food retailing ;
Odd or missing data: Turnover ;  South Australia ;  Liquor retailing ;
Odd or missing data: Turnover ;  South Australia ;  Other specialised food retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Liquor retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Other specialised food retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Hardware, building and garden supplies retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Household goods retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Footwear and other personal accessory retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Clothing, footwear and personal accessory retailing ;
recalibrate(): Data is partly or completely non-numeric.
Odd or missing data: Turnover ;  Tasmania ;  Department stores ;
Odd or missing data: Turnover ;  Tasmania ;  Other retailing n.e.

### Seasonal adjustment check

In [9]:
def seasonal_check() -> None:
    """Produce alternate seasonal adjustment charts."""

    table = "850101"
    meta1 = meta[meta.Table == table]
    df = abs_dict[table]
    idx = pd.PeriodIndex(df.index, freq="M")
    recentdf = df[idx.year >= (idx[-1].year - 5)]

    for did in meta1[mc.did].unique():

        # extract key data
        data = []
        for seasonal in "Original", "Seasonally Adjusted":
            row = meta1[(meta1[mc.did] == did) & (meta1[mc.stype] == seasonal)]
            id_ = row[mc.id].values[0]
            units = row[mc.unit].values[0]
            data.append(recentdf[id_])
        factor = pd.DataFrame(data[0] / data[1], columns=["Factor"])
        factor["Year"] = cast(pd.PeriodIndex, factor.index).year
        factor["Month"] = cast(pd.PeriodIndex, factor.index).month
        data_table = factor.pivot(values="Factor", index="Month", columns="Year")
        data_table.columns = data_table.columns.astype(str) + " ABS"

        # Let's check how the decomp looks
        # based on a more recent assessment of seasonality
        # NOTE: highly speculative analysis.
        post_covid = data[0][data[0].index >= pd.Period("2021-11", freq="M")]
        decomp = decompose(
            post_covid,
            constant_seasonal=True,
            arima_extend=False,
        )
        if decomp is None:
            continue
        months = (
            decomp["Seasonal Weights"]
            .groupby(cast(pd.PeriodIndex, decomp.index).month)
            .mean()
        )
        data_table["Recent Alternate"] = months

        # chart tidy-up
        title, lfooter = fix_abs_title(did, "")
        title = title.replace("Turnover", "").strip()
        title = "\n" + title if title else title + " - Total"
        adj = 0.02
        ylim = data_table.min().min() - adj, data_table.max().max() + adj

        # plot
        ax = data_table.plot.bar()
        finalise_plot(
            ax,
            title=f"Seasonal factors - Monthly Retail Turnover{title}",
            ylabel="Divisor",
            xlabel="Month",
            rfooter=f"{source}",
            lfooter=f"Australia: {lfooter}",
            ylim=ylim,
            legend={"fontsize": "x-small", "ncol": 2},
            show=SHOW,
        )

        # And an ABS  comparative plot.
        alternate = pd.DataFrame(
            [data[1], decomp["Seasonally Adjusted"]],
            index=["ABS seasonally adjusted", "Alternate seasonal adjustment"],
        ).T
        alternate, units = ra.recalibrate(alternate, units)
        ax = alternate.plot(label="ABS Seasonally Adjusted", lw=2.5)
        finalise_plot(
            ax,
            title=f"Alt. Seas. Adj. - Monthly Retail Turnover{title}",
            ylabel=f"Turnover {units}",
            xlabel=None,
            rfooter=f"{source}",
            lfooter=f"Australia: {lfooter}",
            legend={"fontsize": "x-small", "ncol": 2},
            show=SHOW,
        )


# seasonal_check()

## Finished

In [10]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-05-31 13:30:16

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

conda environment: n/a

Compiler    : Clang 20.1.0 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

mgplot : 0.1.3
typing : 3.10.0.0
pandas : 2.2.3
readabs: 0.0.29

Watermark: 2.5.0



In [11]:
print("Finished")

Finished
